In [11]:
import pandas as pd
import numpy as np
import json
import requests
import re
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
import os
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import BytesIO, StringIO

In [12]:
URL = "https://data.cdc.gov/api/views/hfr9-rurv/rows.csv?accessType=DOWNLOAD"

response = requests.get(URL, verify=False)
if response.status_code == 200:
    # Decode the content and read into DataFrame
    df_raw = pd.read_csv(BytesIO(response.content))
    # Print the first few rows to verify if the data has been read successfully
    print(df_raw.head())
    print(df_raw.columns)
    print(df_raw.shape)
    df_raw.info()
else:
    print("Failed to download the file.")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.cdc.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


                                     RowId  YearStart  YearEnd LocationAbbr  \
0    BRFSS~2022~2022~42~Q03~TMC01~AGE~RACE       2022     2022           PA   
1    BRFSS~2022~2022~46~Q03~TMC01~AGE~RACE       2022     2022           SD   
2    BRFSS~2022~2022~16~Q03~TMC01~AGE~RACE       2022     2022           ID   
3    BRFSS~2022~2022~24~Q03~TMC01~AGE~RACE       2022     2022           MD   
4  BRFSS~2022~2022~55~Q03~TMC01~AGE~GENDER       2022     2022           WI   

   LocationDesc Datasource          Class                     Topic  \
0  Pennsylvania      BRFSS  Mental Health  Frequent mental distress   
1  South Dakota      BRFSS  Mental Health  Frequent mental distress   
2         Idaho      BRFSS  Mental Health  Frequent mental distress   
3      Maryland      BRFSS  Mental Health  Frequent mental distress   
4     Wisconsin      BRFSS  Mental Health  Frequent mental distress   

                                            Question Data_Value_Unit  ...  \
0  Percentage of olde

In [15]:
df_cleaned = df_raw.copy()
df_cleaned = df_raw.drop(columns=  ['LocationAbbr','Data_Value_Footnote_Symbol','Data_Value_Footnote','Datasource','Data_Value_Unit','Data_Value_Alt','StratificationCategory1','StratificationCategoryID1','StratificationID1','StratificationCategoryID2','StratificationID2'])
df_cleaned = df_cleaned.dropna(subset=['Data_Value'])
df_cleaned.info()

df_cleaned['Longitude'] = None
df_cleaned['Latitude'] = None

for index, row in df_cleaned.iterrows():
    geolocation = row['Geolocation']
    if isinstance(geolocation, float):
        continue  # Skip float values
    if geolocation:
        match = re.match(r'POINT \(([-0-9.]+) ([-0-9.]+)\)', geolocation)
        if match:
            longitude = float(match.group(1))
            latitude = float(match.group(2))
            df_cleaned.at[index, 'Longitude'] = longitude
            df_cleaned.at[index, 'Latitude'] = latitude

alzheimer_df = df_cleaned.copy()

alzheimer_df.head(5)


<class 'pandas.core.frame.DataFrame'>
Index: 192808 entries, 3 to 284137
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   RowId                    192808 non-null  object 
 1   YearStart                192808 non-null  int64  
 2   YearEnd                  192808 non-null  int64  
 3   LocationDesc             192808 non-null  object 
 4   Class                    192808 non-null  object 
 5   Topic                    192808 non-null  object 
 6   Question                 192808 non-null  object 
 7   DataValueTypeID          192808 non-null  object 
 8   Data_Value_Type          192808 non-null  object 
 9   Data_Value               192808 non-null  float64
 10  Low_Confidence_Limit     192597 non-null  float64
 11  High_Confidence_Limit    192597 non-null  float64
 12  Stratification1          192808 non-null  object 
 13  StratificationCategory2  156041 non-null  object 
 14  Stratific

,RowId,YearStart,YearEnd,LocationDesc,Class,Topic,Question,DataValueTypeID,Data_Value_Type,Data_Value,...,Stratification1,StratificationCategory2,Stratification2,Geolocation,ClassID,TopicID,QuestionID,LocationID,Longitude,Latitude
3,BRFSS~2022~2022~24~Q03~TMC01~AGE~RACE,2022,2022,Maryland,Mental Health,Frequent mental distress,Percentage of older adults who are experiencin...,PRCTG,Percentage,9.0,...,65 years or older,Race/Ethnicity,"Black, non-Hispanic",POINT (-76.60926011 39.29058096),C05,TMC01,Q03,24,-76.60926,39.290581
4,BRFSS~2022~2022~55~Q03~TMC01~AGE~GENDER,2022,2022,Wisconsin,Mental Health,Frequent mental distress,Percentage of older adults who are experiencin...,PRCTG,Percentage,5.6,...,65 years or older,Gender,Male,POINT (-89.81637074 44.39319117),C05,TMC01,Q03,55,-89.816371,44.393191
6,BRFSS~2022~2022~40~Q03~TMC01~AGE~RACE,2022,2022,Oklahoma,Mental Health,Frequent mental distress,Percentage of older adults who are experiencin...,PRCTG,Percentage,21.5,...,Overall,Race/Ethnicity,Native Am/Alaskan Native,POINT (-97.52107021 35.47203136),C05,TMC01,Q03,40,-97.52107,35.472031
7,BRFSS~2022~2022~42~Q03~TMC01~AGE~RACE,2022,2022,Pennsylvania,Mental Health,Frequent mental distress,Percentage of older adults who are experiencin...,PRCTG,Percentage,10.0,...,Overall,Race/Ethnicity,"White, non-Hispanic",POINT (-77.86070029 40.79373015),C05,TMC01,Q03,42,-77.8607,40.79373
8,BRFSS~2022~2022~42~Q46~TOC10~AGE~GENDER,2022,2022,Pennsylvania,Overall Health,"Disability status, including sensory or mobili...",Percentage of older adults who report having a...,PRCTG,Percentage,39.9,...,Overall,Gender,Female,POINT (-77.86070029 40.79373015),C01,TOC10,Q46,42,-77.8607,40.79373


In [16]:
# Creating Topic Dimension

# Mapping dictionary
topic_mapping = {
    'TOC11': 'Arthritis among older adults',
    'TAC03': 'Binge drinking within past 30 days',
    'TSC06': 'Cholesterol checked in past 5 years',
    'TSC02': 'Colorectal cancer screening',
    'TAC01': 'Current smoking',
    'TSC04': 'Diabetes screening within past 3 years',
    'TOC10': 'Disability status, including sensory or mobility limitations',
    'TGC03': 'Duration of caregiving among older adults',
    'TNC01': 'Eating 2 or more fruits daily',
    'TNC02': 'Eating 3 or more vegetables daily',
    'TSC09': 'Ever had pneumococcal vaccine',
    'TGC02': 'Expect to provide care for someone in the next two years',
    'TOC13': 'Fair or poor health among older adults with arthritis',
    'TOC06': 'Fall with injury within last year',
    'TMC01': 'Frequent mental distress',
    'TCC02': 'Functional difficulties associated with subjective cognitive decline or memory loss among older adults',
    'TSC07': 'High blood pressure ever',
    'TSC08': 'Influenza vaccine within past year',
    'TGC04': 'Intensity of caregiving among older adults',
    'TMC03': 'Lifetime diagnosis of depression',
    'TSC01': 'Mammogram within past 2 years',
    'TCC03': 'Need assistance with day-to-day activities because of subjective cognitive decline or memory loss',
    'TNC03': 'No leisure-time physical activity within past month',
    'TNC04': 'Obesity',
    'TOC05': 'Oral health:  tooth retention',
    'TSC03': 'Pap test within past 3 years',
    'TOC01': 'Physically unhealthy days (mean number of days)',
    'TOC09': 'Prevalence of sufficient sleep',
    'TGC01': 'Provide care for a friend or family member in past month',
    'TGC05': 'Provide care for someone with cognitive impairment within the past month',
    'TOC03': 'Recent activity limitations in past month',
    'TOC07': 'Self-rated health (fair to poor health)',
    'TOC08': 'Self-rated health (good to excellent health)',
    'TOC12': 'Severe joint pain among older adults with arthritis',
    'TCC01': 'Subjective cognitive decline or memory loss among older adults',
    'TOC04': 'Taking medication for high blood pressure',
    'TCC04': 'Talked with health care professional about subjective cognitive decline or memory loss',
    'TSC10': 'Up-to-date with recommended vaccines and screenings - Men',
    'TSC11': 'Up-to-date with recommended vaccines and screenings - Women'
}

unique_topic_ids = df_cleaned['TopicID'].unique()
# Converting the array of unique values into a DataFrame
unique_topic_df = pd.DataFrame(unique_topic_ids, columns=['topic_id'])

# Applying the mapping to create a new column with descriptions
unique_topic_df['topic_desc'] = unique_topic_df['topic_id'].map(topic_mapping)
unique_topic_df = unique_topic_df[unique_topic_df['topic_id'] != '<NA>']
unique_topic_df

,topic_id,topic_desc
0,TMC01,Frequent mental distress
1,TOC10,"Disability status, including sensory or mobili..."
2,TOC05,Oral health: tooth retention
3,TSC09,Ever had pneumococcal vaccine
4,TSC10,Up-to-date with recommended vaccines and scree...
5,TSC11,Up-to-date with recommended vaccines and scree...
6,TSC01,Mammogram within past 2 years
7,TSC03,Pap test within past 3 years
8,TNC04,Obesity
9,TSC02,Colorectal cancer screening


In [17]:
# Creating Class Dimension

# Mapping dictionary
class_mapping = {
    'C01': 'Overall Health',
    'C02': 'Nutrition/Physical Activity/Obesity',
    'C03': 'Screenings and Vaccines',
    'C04': 'Smoking and Alcohol Use',
    'C05': 'Mental Health',
    'C06': 'Cognitive Decline',
    'C07': 'Caregiving'
}

unique_class_ids = df_cleaned['ClassID'].unique()
# Converting the array of unique values into a DataFrame
unique_class_df = pd.DataFrame(unique_class_ids, columns=['class_id'])

# Applying the mapping to create a new column with descriptions
unique_class_df['class_desc'] = unique_class_df['class_id'].map(class_mapping)
unique_class_df = unique_class_df[unique_class_df['class_id'] != '<NA>']
unique_class_df

,class_id,class_desc
0,C05,Mental Health
1,C01,Overall Health
2,C03,Screenings and Vaccines
3,C02,Nutrition/Physical Activity/Obesity
4,C04,Smoking and Alcohol Use
5,C06,Cognitive Decline
6,C07,Caregiving


In [18]:
# Creating Question Dimension

# Mapping dictionary
question_mapping = {
    'Q01': 'Percentage of older adults who are eating 2 or more fruits daily',
    'Q02': 'Percentage of older adults who are eating 3 or more vegetables daily',
    'Q03': 'Percentage of older adults who are experiencing frequent mental distress',
    'Q04': 'Percentage of older adults who have been told they have high blood pressure who report currently taking medication for their high blood pressure',
    'Q05': 'Percentage of older adults who have fallen and sustained an injury within last year',
    'Q07': 'Percentage of older adults who report having lost 5 or fewer teeth due to decay or gum disease',
    'Q08': 'Physically unhealthy days (mean number of days in past month)',
    'Q09': 'Percentage of at risk adults (have diabetes, asthma, cardiovascular disease or currently smoke) who ever had a pneumococcal vaccine',
    'Q10': 'Percentage of older adult men who are up to date with select clinical preventive services',
    'Q11': 'Percentage of older adult women who are up to date with select clinical preventive services',
    'Q12': 'Percentage of older adult women who have received a mammogram within the past 2 years',
    'Q13': 'Percentage of older adults who are currently obese, with a body mass index (BMI) of 30 or more',
    'Q14': 'Percentage of older adults who had a cholesterol screening within the past 5 years',
    'Q15': 'Percentage of older adults who had either a home blood stool test within the past year or a sigmoidoscopy or colonoscopy within the past 10 years',
    'Q16': 'Percentage of older adults who have not had any leisure time physical activity in the past month',
    'Q17': 'Percentage of older adults who have smoked at least 100 cigarettes in their entire life and still smoke every day or some days',
    'Q18': 'Percentage of older adults who reported influenza vaccine within the past year',
    'Q19': 'Percentage of older adults without diabetes who reported a blood sugar or diabetes test within 3 years',
    'Q20': 'Percentage of older adult women with an intact cervix who had a Pap test within the past 3 years',
    'Q21': 'Percentage of older adults who reported binge drinking within the past 30 days',
    'Q22': 'Percentage of older adults who have ever been told by a health professional that they have high blood pressure',
    'Q27': 'Percentage of older adults with a lifetime diagnosis of depression',
    'Q30': 'Percentage of older adults who reported subjective cognitive decline or memory loss that is happening more often or is getting worse in the preceding 12 months',
    'Q31': 'Percentage of older adults who reported subjective cognitive decline or memory loss that interferes with their ability to engage in social activities or household chores',
    'Q32': 'Percentage of older adults who self-reported that their health is "fair" or "poor"',
    'Q33': 'Percentage of older adults who self-reported that their health is "good", "very good", or "excellent"',
    'Q34': 'Percentage of older adults getting sufficient sleep (>6 hours)',
    'Q35': 'Mean number of days with activity limitations in the past month',
    'Q36': 'Percentage of older adults who provided care for a friend or family member within the past month',
    'Q37': 'Percentage of older adults currently not providing care who expect to provide care for someone with health problems in the next two years',
    'Q38': 'Percentage of older adults who provided care to a friend or family member for six months or more',
    'Q39': 'Average of 20 or more hours of care per week provided to a friend or family member',
    'Q40': 'Percentage of older adults who provided care for someone with dementia or other cognitive impairment within the past month',
    'Q41': 'Percentage of older adults who reported that as a result of subjective cognitive decline or memory loss that they need assistance with day-to-day activities',
    'Q42': 'Percentage of older adults with subjective cognitive decline or memory loss who reported talking with a health care professional about it',
    'Q43': 'Percentage of older adults ever told they have arthritis',
    'Q44': 'Severe joint pain due to arthritis among older adults with doctor-diagnosed arthritis',
    'Q45': 'Fair or poor health among older adults with doctor-diagnosed arthritis',
    'Q46': 'Percentage of older adults who report having a disability (includes limitations related to sensory or mobility impairments or a physical, mental, or emotional condition)'
}

unique_question_ids = df_cleaned['QuestionID'].unique()
# Converting the array of unique values into a DataFrame
unique_question_df = pd.DataFrame(unique_question_ids, columns=['question_id'])

# Applying the mapping to create a new column with descriptions
unique_question_df['question_desc'] = unique_question_df['question_id'].map(question_mapping)
unique_question_df = unique_question_df[unique_question_df['question_id'] != '<NA>']
unique_question_df

,question_id,question_desc
0,Q03,Percentage of older adults who are experiencin...
1,Q46,Percentage of older adults who report having a...
2,Q07,Percentage of older adults who report having l...
3,Q09,"Percentage of at risk adults (have diabetes, a..."
4,Q10,Percentage of older adult men who are up to da...
5,Q11,Percentage of older adult women who are up to ...
6,Q12,Percentage of older adult women who have recei...
7,Q20,Percentage of older adult women with an intact...
8,Q13,Percentage of older adults who are currently o...
9,Q15,Percentage of older adults who had either a ho...


In [20]:
location_lookup_df = pd.read_csv('CIS9440HW/docs/filtered_location.csv')
location_lookup_df = location_lookup_df.fillna(0)
new_order = ['location_id', 'location_desc','latitude','longitude']
location_lookup_df = location_lookup_df[new_order]
location_lookup_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'CIS9440HW/docs/filtered_location.csv'